In [ ]:
### Deze code bevat de functie merge_annotated_data, die gelabelde data samenvoegt naar één uiteindelijke dataset.
### De output van deze code is een DataFrame met de uiteindelijke dataset.
### This code contains the function merge_annotated_data, which merges annotated documents from the annotators back into one final dataset. 
### The output of this code is a DataFrame with the final dataset.

In [3]:
# inladen benodigde packages
# load necessary packages
import pandas as pd
import numpy as np

In [5]:
def merge_annotated_data(naam_dataset, n_annotators, n_annotations = 3):
    """ functie die de gesplitste data gesplitst door split_annotated_data weer kan samenvoegen. 
    De datasets met de naam naam_dataset_n_annotators.csv worden ingeladen. Vervolgens wordt de oorspronkelijke dataset gereconstrueerd
    uit de ingeladen data. De oorspronkelijke zinnen en het overzicht van de labels wordt gereturned.
    
    Input:
    naam_dataset: str, de naam van de documenen die de annotators hebben ingevuld
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin. default 3
    
    Output:
    merged_data: een DataFrame met daarin de oorspronkelijke data en de n_annotations labels die zijn toegekend.
    """

    """ function that can merge data split by split_annotated_data.
    The datasets with the name naam_dataset_n_annotators.csv are loaded. The original dataset then gets reconstructed
    from the loaded data. The original sentences and the overview of the original labels is returned.

    Input:
    name_dataset: str, the name of the documents that the annotators have filled in
    n_annotators: int, the number of annotators
    n_annotations: int, the number of annotations per sentence. Defaults to 3
    """
    
    # inladen path naar data en creëren lege frames
    # load path to data and create empty frames
    dataset_path = "/data/rstudio/datasets/programma-discriminatie-en-racisme/Datasets_1/"
    merged_data = pd.DataFrame()
    
    for i in range(n_annotators):
        # inladen annotated documenten en samenvoegen
        # load annotated documents and compile
        annotated_doc = pd.read_csv(dataset_path + naam_dataset + "_" + str(i) + '.csv', sep = ',')
        columns = list(annotated_doc.columns)
        
        # document opstellen
        # create document
        if i == 0:
            merged_data = annotated_doc
        else:
            merged_data = pd.concat([merged_data, annotated_doc])
            
    merged_data = merged_data.groupby(['Unnamed: 0']).agg({'id': 'last', 'document': 'last', 'text': 'last', 'zin': 'last', 'score': 'mean'})
    merged_data = merged_data.sort_values(by = ['Unnamed: 0'])
    
    return pd.DataFrame(merged_data)

merged = merge_annotated_data('te_labellen', n_annotators = 5, n_annotations = 3)
display(merged)

KeyError: "Column(s) ['document', 'score'] do not exist"

In [50]:
# output checken
# check output of code
print(list(merged[0]))

[0, 0, 'Beslisnota bij Kamerbrief over Geografische Balans Subsidieregeling en Beleidsintensiveringen Slavernijverleden', '                                                                                    TER BESLUITVORMING\n                                                                                    Nota actief openbaar\n                                                                                    Ja\n                                                                                    Onze referentie\nAan                      StasBZK en MBZK                                            2024-0000015654\nVan                      Programma <mark style="background: #00ced1!important">slavernijverleden</mark>                                Datum\n                                                                                    9 januari 2024\n                                                                                    Opgesteld door\n                          (0) Nota 